In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st 
reload(ap)

<module 'adversarial_perturbations' from '/Users/jordanm/grad/mister_ed/adversarial_perturbations.pyc'>

In [2]:
""" Goal here is to make sure adversarial training works under the refactored model. It _should_, but it might need 
    a few tweaks 
"""

# Load up dataLoader, classifier, normer 
use_gpu = torch.cuda.is_available()
classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                           use_gpu=use_gpu)
classifier_net.eval()
train_loader = cifar_loader.load_cifar_data('train', normalize=False, 
                                            batch_size=16, use_gpu=use_gpu)
val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          batch_size=4, use_gpu=use_gpu)

cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                             std=config.CIFAR10_STDS)

examples, labels = next(iter(val_loader))


Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Make a threat model and attack object 
reload(advtrain)
reload(aar)
delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                              ap.PerturbationParameters(lp_style='inf',
                                                        lp_bound=8.0 / 255.0))
loss_fxn = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer) # USE A PLF LOSS FXN 
fgsm_attack = aar.FGSM(classifier_net, cifar_normer, delta_threat, loss_fxn)
attack_params = advtrain.AdversarialAttackParameters(fgsm_attack, 0.5, 
                                                     attack_specific_params={'attack_kwargs': {'step_size': 0.1}})
print attack_params

rot_threat = ap.ThreatModel(ap.ParameterizedXformAdv, 
                            ap.PerturbationParameters(lp_style='inf', 
                                                      lp_bound=10.0 / 360,
                                                      xform_class=st.RotationTransform))
loss_fxn_rot = plf.VanillaXentropy(classifier_net, normalizer=cifar_normer) # USE A PLF LOSS FXN 
rot_attack = aar.PGD(classifier_net, cifar_normer, rot_threat, loss_fxn_rot)
rot_attack_params = advtrain.AdversarialAttackParameters(rot_attack, 1.0, 
                                                     attack_specific_params={'attack_kwargs': {'step_size': 0.01}})



In [4]:
print isinstance(attack_params, advtrain.AdversarialAttackParameters)

True


In [5]:
reload(advtrain)
            
classifier_net.train()
train_obj = advtrain.AdversarialTraining(classifier_net, cifar_normer, 'refactor_test', 'resnet32')
train_loss = nn.CrossEntropyLoss() # USE A PREBUILT TRAIN LOSS FXN 
train_obj.train(train_loader, 2, train_loss, attack_parameters=[attack_params], verbosity="snoop", 
                adversarial_save_dir='foobar', regularize_adv_scale=1.0)



(Post FGSM):  0.0 correct
[1,     1] accuracy: (0.000, 100.000)
[1,     1] loss: 0.001391
(Post FGSM):  0.0 correct
[1,     2] accuracy: (0.000, 100.000)
[1,     2] loss: 0.002157
(Post FGSM):  37.5 correct
[1,     3] accuracy: (37.500, 100.000)
[1,     3] loss: 0.001262
(Post FGSM):  25.0 correct
[1,     4] accuracy: (25.000, 100.000)
[1,     4] loss: 0.001397
(Post FGSM):  12.5 correct
[1,     5] accuracy: (12.500, 100.000)
[1,     5] loss: 0.001979
(Post FGSM):  37.5 correct
[1,     6] accuracy: (37.500, 100.000)


Process Process-8:
Process Process-7:
Process Process-6:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/py

KeyboardInterrupt: 